In [1]:
import os
import sys
import matplotlib
import matplotlib.pyplot as plt

sys.path.insert(0, '/Users/angela/src/private/bias-in-datasets/active_learning/src')
from Plotter import format_plot, write_file


In [2]:
def plot_eval(experiment_dir, experiment_names, plot_dir, batch_size=128, log_every=100):
    for experiment_name in experiment_names:
        val_file = os.path.join(experiment_dir, experiment_name, "val_eval.txt")
        xs = []
        losses = []
        accs = []
        print("parsing...")
        with open(val_file) as f:
            for i, line in enumerate(f):
                if "Crossent" in line:
                    continue
                vals = line.rstrip().split(" ")
                crossent = float(vals[0])
                acc = float(vals[1]) * 100
                losses.append(crossent)
                accs.append(acc)
                x = i * batch_size * log_every / 1000000.
                xs.append(x)
        print("plotting...")
        #plt.plot(xs, losses, label=experiment_name)
        plt.plot(xs, accs, label=experiment_name)
    with open("/Users/angela/tmp/Test Accuracy-181115_seedednew_aug-1.0") as f:
        xs = []
        ys = []
        for line in f:
            vals = line.rstrip().split(",")
            xs.append(float(vals[0]))
            ys.append(float(vals[1]))
        plt.plot(xs, ys, label="Our Baseline")
    #plot_prefix = "{}/{}_val_losses".format(plot_dir, experiment_name)
    plot_prefix = "{}/{}_val_accs".format(plot_dir, experiment_name)
    format_plot("Num Images Trained (millions)", "Accuracies")
    write_file(plot_prefix)


In [3]:
plot_home_dir = "../plots"
experiment_dir = "../data/output/importance_sampling"
experiment_names = ["181128_oraclegnorm", "181128_uniform"]
plot_dir = os.path.join(plot_home_dir, "181128_reproduce")
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)
    
plot_eval(experiment_dir, experiment_names, plot_dir)

parsing...
plotting...
parsing...
plotting...
../plots/181128_reproduce/181128_uniform_val_accs.pdf


<Figure size 432x288 with 0 Axes>